In [1]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.mllib.tree import RandomForest
from pyspark.ml import Pipeline

from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier



In [2]:
IS_SPARK_SUBMIT_CLI = False
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

In [3]:
if IS_SPARK_SUBMIT_CLI:
    coviddeath = spark.read.csv('uscasestemp1.csv', inferSchema=True, header=True)
else:
    coviddeath = spark.sql("SELECT * FROM uscasestemp1_csv")

In [4]:
data = coviddeath.select("Year","Date","Day", "Temp","Lat","Long","Admin2","Province",col("Case").alias("label"))
data = StringIndexer(inputCol='Admin2', outputCol='Admin2'+"_index").fit(data).transform(data)
data = StringIndexer(inputCol='Province', outputCol='Province'+"_index").fit(data).transform(data)
data.show(5)


+----+----+---+----+-----------+------------+-----------+----------+-----+------------+--------------+
Year|Date|Day|Temp| Lat| Long| Admin2| Province|label|Admin2_index|Province_index|
+----+----+---+----+-----------+------------+-----------+----------+-----+------------+--------------+
2020| 22| 1| 61|34.30828379|-118.2282411|Los Angeles|California| 0| 3.0| 0.0|
2020| 23| 1| 59|34.30828379|-118.2282411|Los Angeles|California| 0| 3.0| 0.0|
2020| 24| 1| 63|34.30828379|-118.2282411|Los Angeles|California| 0| 3.0| 0.0|
2020| 25| 1| 64|34.30828379|-118.2282411|Los Angeles|California| 0| 3.0| 0.0|
2020| 26| 1| 58|34.30828379|-118.2282411|Los Angeles|California| 1| 3.0| 0.0|
+----+----+---+----+-----------+------------+-----------+----------+-----+------------+--------------+
only showing top 5 rows

In [5]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)


Training Rows: 302 Testing Rows: 134

In [6]:
from pyspark.ml.classification import RandomForestClassifier
assembler = VectorAssembler(inputCols =["Day","Temp","Lat","Province_index","Admin2_index"],outputCol="normfeatures")
minMax = MinMaxScaler(inputCol = assembler.getOutputCol(), outputCol="nfeatures")
featVect = VectorAssembler(inputCols=["nfeatures"], outputCol="features")
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
pipeline = Pipeline(stages=[assembler,minMax,featVect,lr])

In [7]:
piplineModel = pipeline.fit(train)
print("Pipeline complete!")


Pipeline complete!

In [8]:
# piplineModel with train data set applies test data set and generate predictions
prediction = piplineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+------------------------------------------------------------------------------------+----------+---------+
features |prediction|trueLabel|
+------------------------------------------------------------------------------------+----------+---------+
[0.25,0.29411764705882354,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |0 |
[0.5,0.6078431372549019,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |0 |
[0.75,0.5294117647058824,0.48516958731189325,1.0,0.3333333333333333] |0.0 |47439 |
[1.0,0.7058823529411765,0.0,0.0,1.0] |0.0 |24262 |
[0.25,0.21568627450980393,1.0,0.3333333333333333,0.0] |0.0 |0 |
[0.25,0.3137254901960784,0.48516958731189325,1.0,0.3333333333333333] |0.0 |0 |
[0.25,0.49019607843137253,0.0,0.0,1.0] |0.0 |1 |
[0.5,0.5686274509803921,0.48516958731189325,1.0,0.3333333333333333] |0.0 |1 |
[0.75,0.3333333333333333,1.0,0.3333333333333333,0.0] |0.0 |182 |
[0.75,0.5686274509803921,0.48516958731189325,1.0,0.3333333333333333] |0.0 |51809 |
[0.75,0.6078431372549019,0.0,0.0,1.0] |0.0 |4045 |
[1.0,0.7058823529411765,0.0,0.0,1.0] |0.0 |24894 |
[0.25,0.1568627450980392,1.0,0.3333333333333333,0.0] |0.0 |0 |
[0.5,0.6078431372549019,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |0 |
[1.0,0.5882352941176471,0.0,0.0,1.0] |0.0 |25699 |
[0.25,0.6274509803921569,0.48516958731189325,1.0,0.3333333333333333] |0.0 |0 |
[0.5,0.4117647058823529,1.0,0.3333333333333333,0.0] |0.0 |0 |
[0.75,0.39215686274509803,1.0,0.3333333333333333,0.0] |0.0 |209 |
[1.0,0.7647058823529411,0.0,0.0,1.0] |0.0 |26225 |
[1.0,0.8823529411764706,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |16040 |
[0.25,0.5294117647058824,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |0 |
[0.5,0.47058823529411764,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |0 |
[0.75,0.4117647058823529,1.0,0.3333333333333333,0.0] |0.0 |219 |
[0.75,0.47058823529411764,0.0,0.0,1.0] |0.0 |5955 |
[0.75,0.7058823529411765,0.48516958731189325,1.0,0.3333333333333333] |0.0 |67551 |
[0.75,0.7450980392156863,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |3189 |
[0.25,0.17647058823529413,1.0,0.3333333333333333,0.0] |0.0 |0 |
[0.25,0.23529411764705882,0.48516958731189325,1.0,0.3333333333333333] |0.0 |0 |
[0.5,0.3137254901960784,0.48516958731189325,1.0,0.3333333333333333] |0.0 |11 |
[0.5,0.49019607843137253,1.0,0.3333333333333333,0.0] |0.0 |0 |
[0.75,0.5882352941176471,1.0,0.3333333333333333,0.0] |0.0 |222 |
[0.75,0.7254901960784313,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |3728 |
[1.0,0.8235294117647058,0.0,0.0,1.0] |0.0 |28665 |
[0.25,0.6274509803921569,0.0,0.0,1.0] |0.0 |1 |
[0.5,0.5294117647058824,0.0,0.0,1.0] |0.0 |14 |
[0.75,0.6470588235294118,1.0,0.3333333333333333,0.0] |0.0 |222 |
[0.75,0.7254901960784313,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |4272 |
[1.0,0.7450980392156863,0.48516958731189325,1.0,0.3333333333333333] |0.0 |180216 |
[0.25,0.6274509803921569,0.0,0.0,1.0] |0.0 |1 |
[0.5,0.6078431372549019,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |0 |
[0.75,0.6862745098039216,0.48516958731189325,1.0,0.3333333333333333] |0.0 |81803 |
[0.75,0.9019607843137255,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |4777 |
[0.25,0.17647058823529413,1.0,0.3333333333333333,0.0] |0.0 |0 |
[0.5,0.6470588235294118,0.0,0.0,1.0] |0.0 |14 |
[0.75,0.5882352941176471,0.48516958731189325,1.0,0.3333333333333333] |0.0 |87028 |
[0.75,0.7450980392156863,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |5029 |
[1.0,0.6078431372549019,0.0,0.0,1.0] |0.0 |31241 |
[0.25,0.29411764705882354,1.0,0.3333333333333333,0.0] |0.0 |0 |
[0.25,0.35294117647058826,0.48516958731189325,1.0,0.3333333333333333] |0.0 |0 |
[0.5,0.6470588235294118,0.0,0.0,1.0] |0.0 |20 |
[0.75,0.4117647058823529,0.4277898629332098,0.6666666666666666,0.6666666666666666] |0.0 |5521 |
[0.25,0.43137254901960786,0.4277898629332098,0.6666666666666666,0.6666666666666666]

In [9]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluation = MulticlassClassificationEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluation.evaluate(prediction)
print("Accuracy of Logistic Regression is: ",accuracy)

Accuracy of Logistic Regression is: 0.3656716417910448

In [10]:
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.634328